In [1]:
import json
import csv
import time
from datetime import datetime, timedelta
from numpy import array
import numpy as np
import pandas as pd

In [2]:
symbols = ["OTO", "HPT", "AMANAH"]
items = ["2017-02-14", "2017-02-15", "2017-02-16"]
# items = ["2017-02-28"]

In [3]:
def getFreefloat():
    allShare = 250000000  # ACAP
    freeFloat = 67.03
    return (allShare * freeFloat)/100

def check(list):
    if list[1] is not 0 and list[2] is not 0 and list[4] is 0: return 0
    else: return 1

def getSumVol(list):
    # Sum = list[0] + list[1] + list[2] + list[3] + list[4]
    Sum = np.sum(list)
    return Sum

def event(eve):
    if eve == "B":
        return 1
    elif eve == "S":
        return 0
    else:
        return 5

def getTime(time):
    timeSplit = time.split(':')
    return timeSplit

def getDate(Date):
    dateSplit = Date.split('-')
    return dateSplit


def Timestamp2Datetime(Timestamp):
    Datetime = datetime.fromtimestamp(Timestamp-25200).strftime('%Y-%m-%d-%H-%M-%S')
    # print Datetime
    return Datetime

def Datetime2Timestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    # return td.total_seconds()
    return (td.microseconds + (td.seconds + td.days * 86400) * 10**6) // 10**6


def toTemp(price, vol, eve, symbol):
    sumVO = getSumVol(vol)
    compareBid1[symbol] = tempBid[symbol][5]
    compareOff1[symbol] = tempOffer[symbol][5]


    if eve == 1:

        tempBid[symbol] = price + vol

        difBO[symbol] = sumVO - tempBidVol[symbol]

        if difBO[symbol] <= -1000000:
            volBO[symbol] = -4
        elif -1000000 < difBO[symbol] <= -500000:
            volBO[symbol] = -3
        elif -500000 < difBO[symbol] <= -100000:
            volBO[symbol] = -2
        elif -100000 < difBO[symbol] <= -100:
            volBO[symbol] = -1
        elif -100 < difBO[symbol] <= 100000:
            volBO[symbol] = 1
        elif 100000 < difBO[symbol] <= 500000:
            volBO[symbol] = 2
        elif 500000 < difBO[symbol] <= -1000000:
            volBO[symbol] = 3
        elif difBO[symbol] > 1000000:
            volBO[symbol] = 4

        tempBidVol[symbol] = sumVO
        return price + vol + tempOffer[symbol] + [tempBidVol[symbol]] + [tempOffVol[symbol]]
    else:

        tempOffer[symbol] = price + vol

        difBO[symbol] = sumVO - tempOffVol[symbol]

        if difBO[symbol] <= -1000000:
            volBO[symbol] = -40
        elif -1000000 < difBO[symbol] <= -500000:
            volBO[symbol] = -30
        elif -500000 < difBO[symbol] <= -100000:
            volBO[symbol] = -20
        elif -100000 < difBO[symbol] <= -100:
            volBO[symbol] = -10
        elif -100 < difBO[symbol] <= 100000:
            volBO[symbol] = 10
        elif 100000 < difBO[symbol] <= 500000:
            volBO[symbol] = 20
        elif 500000 < difBO[symbol] <= -1000000:
            volBO[symbol] = 30
        elif difBO[symbol] > 1000000:
            volBO[symbol] = 40

        tempOffVol[symbol] = sumVO
        return tempBid[symbol] + price + vol + [tempBidVol[symbol]] + [tempOffVol[symbol]]



def checkTimestamp(timestamp,symbol):
    if timestamp < tempTimestamp[symbol]:
        return tempTimestamp[symbol],1
    else:
        tempTimestamp[symbol] = timestamp
        return timestamp,0










def getSpread(x):
    ''' x<2         0.01    0
        2<=x<5       0.02    1
        5<=x<10      0.05    2
        10<=x<25     0.1     3
        25<=x<100    0.25    4
        100<=x<200   0.5     5
        200<=x<400   1.0     6
        x>=400       2.0     7
    '''
    if x < 200:
        return 0, 1, 200
    elif 200 <= x < 500:
        return 1, .5, 500
    elif 500 <= x < 1000:
        return 2, .2, 1000
    elif 1000 <= x < 2500:
        return 3, .1, 2500
    elif 2500 <= x < 10000:
        return 4, .04, 10000
    elif 10000 <= x < 20000:
        return 5, .02, 20000
    elif 20000 <= x < 40000:
        return 6, .01, 40000
    else:
        return 7, .005, 40000

def getDifSpread(x1, x2):
    a1, t1, z1 = getSpread(x1)
    a2, t2, z2 = getSpread(x2)
    if a1 == a2:
        Dif = (x2 - x1) * t1
    elif a2 > a1:
        Dif = ((x2 - z1) * t2) + ((z1 - x1) * t1)
    else:
        Dif = -(((x1 - z2) * t1) + ((z2 - x2) * t2))
    return int(Dif)



def calmoney(pbid0,pbid1,pbid2,pbid3,pbid4,vbid0,vbid1,vbid2,vbid3,vbid4):
    money = ((pbid0*vbid0)+(pbid1*vbid1)+(pbid2*vbid2)+(pbid3*vbid3)+(pbid4*vbid4))/100
    return money

In [4]:
idSymbol = dict()
tempBid = dict()
tempOffer = dict()
tempBidVol = dict()
tempOffVol = dict()
tempAll = dict()
priATO = dict()
volATO = dict()
tempTimestamp = dict()
marketStatus = dict()
forTemp = dict()
clearNoise = dict()

sumOrder = dict()
countOrder = dict()

lastPrice = dict()
prior = dict()
highPrice = dict()
lowPrice = dict()
avgPrice = dict()
tradeVol = dict()
buyVol = dict()
sellVol = dict()
auctVol = dict()

spread = dict()
count100k = dict()
difBO = dict()

tf30 = dict()
volBO = dict()
typeBO = dict()
compareBid1 = dict()
compareOff1 = dict()


allbidmoney = dict()
sidetransac = dict()

dfall = dict()



stock =dict()
stock1 =dict()
ato = dict()

for symbol in symbols:
    # tempBid[symbol] = [None, None, None, None, None, None, None, None, None, None]
    # tempOffer[symbol] = [None, None, None, None, None, None, None, None, None, None]
    tempBid[symbol] = [0,0,0,0,0,0,0,0,0,0]
    tempOffer[symbol] = [0,0,0,0,0,0,0,0,0,0]
    # forTemp[symbol] = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]

    priATO[symbol] = 0
    volATO[symbol] = 0
    tempTimestamp[symbol] = 0
    marketStatus[symbol] = 0
    clearNoise[symbol] = 0
    idSymbol[symbol] = 0

    sumOrder[symbol] = 0
    countOrder[symbol] = 0

    lastPrice[symbol] = 0
    prior[symbol] = 0
    highPrice[symbol] = 0
    lowPrice[symbol] = 0
    avgPrice[symbol] = 0
    tradeVol[symbol] = 0
    buyVol[symbol] = 0
    sellVol[symbol] = 0
    auctVol[symbol] = 0

    spread[symbol] = 0
    count100k[symbol] = 0
    difBO[symbol] =0
    tf30[symbol] = 0
    volBO[symbol] = 0
    typeBO[symbol] = 0

    tempBidVol[symbol] = 0
    tempOffVol[symbol] = 0
    compareBid1[symbol] = 0
    compareOff1[symbol] = 0

    allbidmoney[symbol] = 0
    sidetransac[symbol] = 0
    
    stock[symbol] = pd.DataFrame()
    stock1[symbol] = pd.DataFrame()
    ato[symbol] = pd.DataFrame()
    


for symbol in symbols:
    forTemp[symbol] = []

In [5]:
t0 = time.time()

# ref_files = [open("TESTIPython\\List\\" + Symbol + ".csv", "a") for Symbol in symbols]
# ref_files2 = [open("TESTIPython\\Listato\\" + Symbol + ".csv", "a") for Symbol in symbols]



three_tuple = [(current, items[idx - 1] if idx >= 1 else None, items[idx + 1] if idx < len(items) - 1 else None) for idx, current in enumerate(items)]
# return a 3-tuple with current, previous and next elements

    

for enum, Date in enumerate(items):

    yesterday = three_tuple[enum][1]
    today = three_tuple[enum][0]
    tomorrow = three_tuple[enum][2]
    
    YYYY,MM,DD = getDate(Date)

    YYYY = int(YYYY)
    MM = int(MM)
    DD = int(DD)
    
    with open("INPUT\\" +Date+".dat") as f:

        content = f.readlines()
        for line in content:
            if 'data' in line:
                try:
                    jsonData = line.replace("data: ", "")
                    jsonDecoded = json.loads(jsonData)
                except ValueError:
                    # print Date, jsonData
                    continue

                if 'sym' in jsonDecoded.keys():
                    sym = jsonDecoded["sym"]
                    if sym in symbols:
                        if 'isf' in jsonDecoded.keys():

                            pri = jsonDecoded["pri"]  # price
                            priATO[sym] = pri

                            tim = jsonDecoded["tim"]    #time
                            HH, mm, SS = getTime(jsonDecoded["tim"])            ###
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)
                            Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
                            Timestamp = checkTimestamp(Timestamp,sym)

                            vol = jsonDecoded["vol"]  # volume
                            volATO[sym] = vol

                            op1 = jsonDecoded["op1"]  # open1
                            op2 = jsonDecoded["op2"]  # open2

                            isf = jsonDecoded["isf"]
                            if isf == 'F':
                                marketStatus[sym] = 0   #Pre-Open1
                                if HH == 14:
                                    marketStatus[sym] = 2   #Pre-Open2
                                if HH == 16:
                                    marketStatus[sym] = 3  # Pre-Open2

                            elif isf == 'T':
                                marketStatus[sym] = 1   #Open
                                clearNoise[sym] = Timestamp[0]

                                if HH == 16 and mm >30:
                                    marketStatus[sym] = 4   #Pre-Close
                            else:
                                marketStatus[sym] = 5
                                print "Unknown status"



#                                 table = [marketStatus[sym]] + [Timestamp[0]] + [Date] + [tim] + [pri] + [vol] + [op1] + [op2]
#                                 df2 = pd.DataFrame(table)
#                                 ato[sym] = ato[sym].append(df2, ignore_index=True)

                            ato[sym] = ato[sym].append({'marketStatus':marketStatus[sym], 'Timestamp':Timestamp[0], 'Date':Date, 'time':tim, 'pri':pri, 'vol':vol, 'op1':op1, 'op2':op2}, ignore_index=True)

                        elif 'time' in jsonDecoded.keys():
                            pri = jsonDecoded["pri"]
                            if check(pri) is 0:
                                continue

                            eve = event(jsonDecoded["sid"])

                            tim = jsonDecoded["time"]
                            HH, mm, SS = getTime(jsonDecoded["time"])  ###
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)

                            Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
                            Timestamp = checkTimestamp(Timestamp,sym)

                            tempY, tempM, tempD, HH, mm, SS = getDate(Timestamp2Datetime(Timestamp[0]))
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)



                            ids = jsonDecoded["id"]  ##identify number of stock
                            idSymbol[sym] = ids

                            vol = jsonDecoded["vol"]

                            # if jsonDecoded["sid"] == 'S':
                            #     side = 1  # Fill Offer
                            # elif jsonDecoded["sid"] == 'B':
                            #     side = 0  # Fill Bid
                            # else:
                            #     side = 3

                            side = event(jsonDecoded["sid"])

                            bidOffer = toTemp(pri, vol, eve, sym)


                            allbidmoney[sym] = calmoney(bidOffer[0],bidOffer[1],bidOffer[2],bidOffer[3],bidOffer[4],bidOffer[5],bidOffer[6],bidOffer[7],bidOffer[8],bidOffer[9])




                            if bidOffer[0] == 0 and marketStatus[sym] != 1:
                                bidOffer[0] = priATO[sym]
                            if bidOffer[10] == 0 and marketStatus[sym] != 1:
                                bidOffer[10] = priATO[sym]

                            if marketStatus[sym] == 1:
                                spread[sym] = getDifSpread(prior[sym],bidOffer[0])

                                if HH < 10:
                                    tf30[sym] = 0
                                elif HH == 10 and mm < 30:
                                    tf30[sym] = 1
                                elif HH == 10 and mm >= 30:
                                    tf30[sym] = 2
                                elif HH == 11 and mm < 30:
                                    tf30[sym] = 3
                                elif HH == 11 and mm >= 30:
                                    tf30[sym] = 4
                                elif HH == 12 and mm <= 30:
                                    tf30[sym] = 5
                                elif HH == 14 and mm < 30:
                                    tf30[sym] = 6
                                elif HH == 14 and mm >= 30:
                                    tf30[sym] = 7
                                elif HH == 15 and mm < 30:
                                    tf30[sym] = 8
                                elif HH == 15 and mm >= 30:
                                    tf30[sym] = 9
                                elif HH == 16 and mm <= 30:
                                    tf30[sym] = 10


                            # if side == 1:   #B(Bid)=1, S(Offer)=0
                            #     if sumOrder[sym] == 0:
                            #         if bidOffer[5] != tempBid[sym][5]:
                            #             typeBO = 0
                            #         else:
                            #
                            # if bidOffer[15] != tempOffer[sym][5]:
                            if sumOrder[sym] != 0:
                                volBO[sym] = sumOrder[sym]
                                # typeBO[sym] = side
                                if sidetransac[sym] == 0:
                                    typeBO[sym] = 'Sell'
                                else:
                                    typeBO[sym] = 'Buy'
                            else:
                                if side == 1:
                                    if compareBid1[sym] == bidOffer[5]:
                                        typeBO[sym] = 'Bid Not slot 0'
                                    else:
                                        typeBO[sym] = 'Bid is slot 0'
                                else:
                                    if compareOff1[sym] == bidOffer[15]:
                                        typeBO[sym] = 'Offer Not slot 0'
                                    else:
                                        typeBO[sym] = 'Offer is slot 0'

                            # a = [marketStatus[sym]] + [id] + [Timestamp[0]]+ [Timestamp[1]] + [side]+ [Date] + [tim] + bidOffer

                            # forTemp[sym] = [Date] + [tim] + [idSymbol[sym]] + [marketStatus[sym]] + [Timestamp[0]]+ [Timestamp[1]] + [side] + bidOffer + \
                            #                [volATO[sym]] + [sumOrder[sym]] + [countOrder[sym]] + \
                            #                [tradeVol[sym]] + [lastPrice[sym]] + [prior[sym]] + [highPrice[sym]] + [lowPrice[sym]] + [avgPrice[sym]] + \
                            #                [gain[sym]] + [count100k[sym]] + [buyVol[sym]] + [sellVol[sym]] + [auctVol[sym]]



#                                 forTemp[sym] = [tf30[sym]] + [Date] + [tim] + [HH] + [mm] + [SS] + [idSymbol[sym]]  + [Timestamp[0]] + [Timestamp[1]] + bidOffer + \
#                                                [difBO[sym]] + [volBO[sym]] + [typeBO[sym]] + [allbidmoney[sym]] +[compareOff1[sym]] + [bidOffer[15]] + [side] + [sumOrder[sym]] + [countOrder[sym]] + \
#                                                [tradeVol[sym]] + [lastPrice[sym]] + [prior[sym]] + [highPrice[sym]] + [lowPrice[sym]] + [avgPrice[sym]] + \
#                                                [spread[sym]] + [count100k[sym]] + [buyVol[sym]] + [sellVol[sym]] + [auctVol[sym]]


                            if clearNoise[sym] == Timestamp[0]:
                                continue

                            if HH == 9 and mm == 30:
                                continue
                            # if marketStatus[sym] == 4:
                            if marketStatus[sym] != 1:  ##selective row when market opened only.
                                continue

#                                     wr = csv.writer(ref_files2[symbols.index(sym)], lineterminator='\n')
#                                     wr.writerow(forTemp[sym])

                                # np.array(forTemp[sym]).dump(open('array2.npy', 'wb'))

                            else:
#                                     wr = csv.writer(ref_files[symbols.index(sym)], lineterminator='\n')
#                                     wr.writerow(forTemp[sym])
                                stock[sym] = stock[sym].append({'tf30':tf30[sym], 'Timestamp':Timestamp[0], 'Date':Date, 'yesterday':yesterday, 'today': today, 'tomorrow':tomorrow, 'time':tim, 'BidOffer':bidOffer, 'difBO':difBO[sym], 'typeBO':typeBO[sym], 'spread':spread[sym]}, ignore_index=True)


                                # sql = "INSERT INTO %s (hour,minute,second,timestamp,bid0,bid1,bid2,bid3,bid4) VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s')" % (
                                #     sym, HH, mm, SS, Timestamp[0], bidOffer[0], bidOffer[1], bidOffer[2],
                                #     bidOffer[3], bidOffer[4])
                                #
                                # try:
                                #     cursor.execute(sql)
                                #     # db.commit()
                                # except Exception as e:
                                #     print e
                                #     db.rollback()

                                # np.array(forTemp[sym]).dump(open('array.npy', 'wb'))

                                sumOrder[sym] = 0
                                countOrder[sym] = 0

                        elif 'ava' in jsonDecoded.keys():
                            tim = jsonDecoded["tim"]  ##time
                            HH, mm, SS = getTime(jsonDecoded["tim"])
                            HH = int(HH)
                            mm = int(mm)
                            SS = int(SS)
                            Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
                            Timestamp = checkTimestamp(Timestamp,sym)

                            actVol = jsonDecoded["vol"]  ##vol action
                            prc = jsonDecoded["prc"]  # Last price action(Baht) must / 100
                            lastPrice[sym] = prc

                            # ava = jsonDecoded["ava"]  # Total Trade Value(Baht) must / 100
                            avo = jsonDecoded["avo"]  # Total Trade Volume(Share)
                            tradeVol[sym] = avo

                            bvo = jsonDecoded["bvo"]  # Net Buy Volume(Share)
                            svo = jsonDecoded["svo"]  # Net Sell Volume(Share)
                            ovo = jsonDecoded["ovo"]  # Net ATO/ATC Volume(Share)

                            buyVol[sym] = bvo
                            sellVol[sym] = svo
                            auctVol[sym] = ovo

                            # sid = jsonDecoded["sid"]  ## B = Buy, S = Sell

                            prr = jsonDecoded["prr"]  # Previous Close must / 100
                            hgh = jsonDecoded["hgh"]  # high Price must / 100
                            low = jsonDecoded["low"]  # low Price must / 100
                            avg = jsonDecoded["avg"]  # Average Price must / 100
                            prior[sym] = prr
                            highPrice[sym] = hgh
                            lowPrice[sym] = low
                            avgPrice[sym] = avg

                            sidetransac[sym] = event(jsonDecoded["sid"])

                            sumOrder[sym] += actVol
                            countOrder[sym] += 1

                            if sumOrder[sym] >= 100000:
                                count100k[sym] = 1
                            else:
                                count100k[sym] = 0
                            
                            

                                
t1 = time.time()
total = t1-t0

print total

749.786000013


In [6]:
stock['HPT']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,tomorrow,typeBO,yesterday
0,"[130, 129, 128, 127, 126, 59800, 134100, 26700...",2017-02-14,1.487066e+09,20000.0,130.0,0.0,09:57:03,2017-02-15,Offer Not slot 0,None
1,"[130, 129, 128, 127, 126, 109800, 134100, 2670...",2017-02-14,1.487066e+09,50000.0,130.0,1.0,10:00:20,2017-02-15,Bid is slot 0,None
2,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,200.0,130.0,1.0,10:02:29,2017-02-15,Bid Not slot 0,None
3,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,100000.0,130.0,1.0,10:02:40,2017-02-15,Bid Not slot 0,None
4,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:02:55,2017-02-15,Offer Not slot 0,None
5,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:10,2017-02-15,Offer Not slot 0,None
6,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:20,2017-02-15,Bid Not slot 0,None
7,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:20,2017-02-15,Offer Not slot 0,None
8,"[130, 129, 128, 127, 126, 50800, 134100, 26720...",2017-02-14,1.487067e+09,-59000.0,0.0,1.0,10:03:22,2017-02-15,Sell,None
9,"[129, 128, 127, 126, 125, 134100, 267200, 1780...",2017-02-14,1.487067e+09,120200.0,-1.0,1.0,09:56:11,2017-02-15,Sell,None


In [54]:
## confused, why dataframes are Null value ##
# t0 = time.time()

# ref_files = [open("TESTIPython\\List\\" + Symbol + ".csv", "a") for Symbol in symbols]
# ref_files2 = [open("TESTIPython\\Listato\\" + Symbol + ".csv", "a") for Symbol in symbols]


# for Date in items:

#     YYYY,MM,DD = getDate(Date)

#     YYYY = int(YYYY)
#     MM = int(MM)
#     DD = int(DD)
    
#     with open("INPUT\\" +Date+".dat") as f:
            
#         content = f.readlines()
#         for line in content:
#             if 'data' in line:
#                 try:
#                     jsonData = line.replace("data: ", "")
#                     jsonDecoded = json.loads(jsonData)
#                 except ValueError:
#                     # print Date, jsonData
#                     continue

#                 if 'sym' in jsonDecoded.keys():
#                     sym = jsonDecoded["sym"]

#                     if sym in symbols:
#                         if 'time' in jsonDecoded.keys():
#                             pri = jsonDecoded["pri"]
#                             if check(pri) is 0:
#                                 continue

#                             eve = event(jsonDecoded["sid"])

#                             tim = jsonDecoded["time"]
#                             HH, mm, SS = getTime(jsonDecoded["time"]) 
#                             HH = int(HH)
#                             mm = int(mm)
#                             SS = int(SS)

#                             Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
#                             Timestamp = checkTimestamp(Timestamp,sym)

#                             tempY, tempM, tempD, HH, mm, SS = getDate(Timestamp2Datetime(Timestamp[0]))
#                             HH = int(HH)
#                             mm = int(mm)
#                             SS = int(SS)



#                             ids = jsonDecoded["id"]  ##identify number of stock
#                             idSymbol[sym] = ids

#                             vol = jsonDecoded["vol"]

#                             # if jsonDecoded["sid"] == 'S':
#                             #     side = 1  # Fill Offer
#                             # elif jsonDecoded["sid"] == 'B':
#                             #     side = 0  # Fill Bid
#                             # else:
#                             #     side = 3

#                             side = event(jsonDecoded["sid"])

#                             bidOffer = toTemp(pri, vol, eve, sym)


#                             allbidmoney[sym] = calmoney(bidOffer[0],bidOffer[1],bidOffer[2],bidOffer[3],bidOffer[4],bidOffer[5],bidOffer[6],bidOffer[7],bidOffer[8],bidOffer[9])




#                             if bidOffer[0] == 0 and marketStatus[sym] != 1:
#                                 bidOffer[0] = priATO[sym]
#                             if bidOffer[10] == 0 and marketStatus[sym] != 1:
#                                 bidOffer[10] = priATO[sym]

#                             if marketStatus[sym] == 1:
#                                 spread[sym] = getDifSpread(prior[sym],bidOffer[0])

#                                 if HH < 10:
#                                     tf30[sym] = 0
#                                 elif HH == 10 and mm < 30:
#                                     tf30[sym] = 1
#                                 elif HH == 10 and mm >= 30:
#                                     tf30[sym] = 2
#                                 elif HH == 11 and mm < 30:
#                                     tf30[sym] = 3
#                                 elif HH == 11 and mm >= 30:
#                                     tf30[sym] = 4
#                                 elif HH == 12 and mm <= 30:
#                                     tf30[sym] = 5
#                                 elif HH == 14 and mm < 30:
#                                     tf30[sym] = 6
#                                 elif HH == 14 and mm >= 30:
#                                     tf30[sym] = 7
#                                 elif HH == 15 and mm < 30:
#                                     tf30[sym] = 8
#                                 elif HH == 15 and mm >= 30:
#                                     tf30[sym] = 9
#                                 elif HH == 16 and mm <= 30:
#                                     tf30[sym] = 10


#                             # if side == 1:   #B(Bid)=1, S(Offer)=0
#                             #     if sumOrder[sym] == 0:
#                             #         if bidOffer[5] != tempBid[sym][5]:
#                             #             typeBO = 0
#                             #         else:
#                             #
#                             # if bidOffer[15] != tempOffer[sym][5]:
#                             if sumOrder[sym] != 0:
#                                 volBO[sym] = sumOrder[sym]
#                                 # typeBO[sym] = side
#                                 if sidetransac[sym] == 0:
#                                     typeBO[sym] = 'Sell'
#                                 else:
#                                     typeBO[sym] = 'Buy'
#                             else:
#                                 if side == 1:
#                                     if compareBid1[sym] == bidOffer[5]:
#                                         typeBO[sym] = 'Bid Not slot 0'
#                                     else:
#                                         typeBO[sym] = 'Bid is slot 0'
#                                 else:
#                                     if compareOff1[sym] == bidOffer[15]:
#                                         typeBO[sym] = 'Offer Not slot 0'
#                                     else:
#                                         typeBO[sym] = 'Offer is slot 0'

#                             # a = [marketStatus[sym]] + [id] + [Timestamp[0]]+ [Timestamp[1]] + [side]+ [Date] + [tim] + bidOffer

#                             # forTemp[sym] = [Date] + [tim] + [idSymbol[sym]] + [marketStatus[sym]] + [Timestamp[0]]+ [Timestamp[1]] + [side] + bidOffer + \
#                             #                [volATO[sym]] + [sumOrder[sym]] + [countOrder[sym]] + \
#                             #                [tradeVol[sym]] + [lastPrice[sym]] + [prior[sym]] + [highPrice[sym]] + [lowPrice[sym]] + [avgPrice[sym]] + \
#                             #                [gain[sym]] + [count100k[sym]] + [buyVol[sym]] + [sellVol[sym]] + [auctVol[sym]]



#                             forTemp[sym] = [tf30[sym]] + [Date] + [tim] + [HH] + [mm] + [SS] + [idSymbol[sym]]  + [Timestamp[0]] + [Timestamp[1]] + bidOffer + \
#                                            [difBO[sym]] + [volBO[sym]] + [typeBO[sym]] + [allbidmoney[sym]] +[compareOff1[sym]] + [bidOffer[15]] + [side] + [sumOrder[sym]] + [countOrder[sym]] + \
#                                            [tradeVol[sym]] + [lastPrice[sym]] + [prior[sym]] + [highPrice[sym]] + [lowPrice[sym]] + [avgPrice[sym]] + \
#                                            [spread[sym]] + [count100k[sym]] + [buyVol[sym]] + [sellVol[sym]] + [auctVol[sym]]


#                             if clearNoise[sym] == Timestamp[0]:
#                                 continue

#                             if HH == 9 and mm == 30:
#                                 continue
#                             # if marketStatus[sym] == 4:
#                             if marketStatus[sym] != 1:  ##selective row when market opened only.
#                                 continue
# #                                 wr = csv.writer(ref_files2[symbols.index(sym)], lineterminator='\n')
# #                                 wr.writerow(forTemp[sym])

# #                                 np.array(forTemp[sym]).dump(open('array2.npy', 'wb'))
#                             else:
#                                 stock[sym] = stock[sym].append({'tf30':tf30[sym], 'Timestamp':Timestamp[0], 'Date':Date, 'time':tim, 'BidOffer':bidOffer, 'difBO':difBO[sym], 'typeBO':typeBO[sym], 'spread[sym]':spread[sym]}, ignore_index=True)
#                                 sumOrder[sym] = 0
#                                 countOrder[sym] = 0
            
        
# #                                     wr = csv.writer(ref_files[symbols.index(sym)], lineterminator='\n')
# #                                     wr.writerow(forTemp[sym])
# #                                 stock1[sym] = stock[sym].append({'tf30':tf30, 'Timestamp':Timestamp[0], 'Date':Date, 'time':tim, 'BidOffer':bidOffer, 'difBO':difBO[sym], 'typeBO':typeBO[sym], 'spread':spread}, ignore_index=True)
                                


#                                 # sql = "INSERT INTO %s (hour,minute,second,timestamp,bid0,bid1,bid2,bid3,bid4) VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s')" % (
#                                 #     sym, HH, mm, SS, Timestamp[0], bidOffer[0], bidOffer[1], bidOffer[2],
#                                 #     bidOffer[3], bidOffer[4])
#                                 #
#                                 # try:
#                                 #     cursor.execute(sql)
#                                 #     # db.commit()
#                                 # except Exception as e:
#                                 #     print e
#                                 #     db.rollback()

#                                 # np.array(forTemp[sym]).dump(open('array.npy', 'wb'))

                                

#                         elif 'ava' in jsonDecoded.keys():
#                             tim = jsonDecoded["tim"]  ##time
#                             HH, mm, SS = getTime(jsonDecoded["tim"])
#                             HH = int(HH)
#                             mm = int(mm)
#                             SS = int(SS)
#                             Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
#                             Timestamp = checkTimestamp(Timestamp,sym)

#                             actVol = jsonDecoded["vol"]  ##vol action
#                             prc = jsonDecoded["prc"]  # Last price action(Baht) must / 100
#                             lastPrice[sym] = prc

#                             # ava = jsonDecoded["ava"]  # Total Trade Value(Baht) must / 100
#                             avo = jsonDecoded["avo"]  # Total Trade Volume(Share)
#                             tradeVol[sym] = avo

#                             bvo = jsonDecoded["bvo"]  # Net Buy Volume(Share)
#                             svo = jsonDecoded["svo"]  # Net Sell Volume(Share)
#                             ovo = jsonDecoded["ovo"]  # Net ATO/ATC Volume(Share)

#                             buyVol[sym] = bvo
#                             sellVol[sym] = svo
#                             auctVol[sym] = ovo

#                             # sid = jsonDecoded["sid"]  ## B = Buy, S = Sell

#                             prr = jsonDecoded["prr"]  # Previous Close must / 100
#                             hgh = jsonDecoded["hgh"]  # high Price must / 100
#                             low = jsonDecoded["low"]  # low Price must / 100
#                             avg = jsonDecoded["avg"]  # Average Price must / 100
#                             prior[sym] = prr
#                             highPrice[sym] = hgh
#                             lowPrice[sym] = low
#                             avgPrice[sym] = avg

#                             sidetransac[sym] = event(jsonDecoded["sid"])

#                             sumOrder[sym] += actVol
#                             countOrder[sym] += 1

#                             if sumOrder[sym] >= 100000:
#                                 count100k[sym] = 1
#                             else:
#                                 count100k[sym] = 0
                                
                                
# t1 = time.time()
# total = t1-t0

# print total

18.3190000057


In [107]:
stock

{'AMANAH':                                                 BidOffer        Date  \
 0      [145, 142, 141, 140, 139, 1700, 176000, 188000...  2017-02-14   
 1      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 2      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 3      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 4      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 5      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 6      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 7      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 8      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 9      [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 10     [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 11     [142, 141, 140, 139, 138, 176000, 188000, 1912...  2017-02-14   
 12     [142, 141, 140, 139, 138, 176000,

In [21]:
# ato1 = ato

In [8]:
ato1['AMANAH'].head()

,Date,Timestamp,marketStatus,op1,op2,pri,time,vol
0,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:21,221800.0
1,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,231800.0
2,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,241800.0
3,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,251800.0
4,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:23,261800.0


In [9]:
ato1['AMANAH'] = ato1['AMANAH'].assign(ln_A = lambda x: np.log(x.vol))
ato1['AMANAH'].head()

,Date,Timestamp,marketStatus,op1,op2,pri,time,vol,ln_A
0,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:21,221800.0,12.309531
1,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,231800.0,12.353630
2,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,241800.0,12.395866
3,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,251800.0,12.436390
4,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:23,261800.0,12.475336


In [13]:
for sym in symbols:
    ato[sym].to_csv("ato\\" +sym+'.csv')
    stock[sym].to_csv("stock\\" +sym+'.csv')

In [117]:
stock['AMANAH']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,typeBO
0,"[145, 142, 141, 140, 139, 1700, 176000, 188000...",2017-02-14,1.487066e+09,-13300.0,5.0,0.0,09:56:23,Sell
1,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,192300.0,2.0,0.0,09:56:13,Bid is slot 0
2,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,-284400.0,2.0,0.0,09:56:28,Offer Not slot 0
3,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,100.0,2.0,0.0,09:56:28,Offer Not slot 0
4,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,-19600.0,2.0,0.0,09:56:29,Buy
5,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,10000.0,2.0,0.0,09:56:30,Offer Not slot 0
6,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,3000.0,2.0,0.0,09:56:32,Offer is slot 0
7,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,40000.0,2.0,0.0,09:56:32,Offer is slot 0
8,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,100.0,2.0,0.0,09:56:35,Offer is slot 0
9,"[142, 141, 140, 139, 138, 176000, 188000, 1912...",2017-02-14,1.487066e+09,5000.0,2.0,0.0,09:56:37,Bid Not slot 0


In [110]:
for sym in symbols:
    for r in range(11):
        print sym, r, stock[sym].loc[stock[sym]['tf30'] == r]['spread'].mean()
    

# stock['OTO'] = stock['OTO'].assign(predict = stock['OTO'].loc[stock['OTO']['tf30'] ==  1]['spread[sym]'])

 OTO 0 0.469387755102
OTO 1 2.33333333333
OTO 2 0.222453222453
OTO 3 -1.26644736842
OTO 4 -4.24401913876
OTO 5 -2.07971014493
OTO 6 nan
OTO 7 -1.18285714286
OTO 8 -1.15527950311
OTO 9 -1.7614213198
OTO 10 -3.03389830508
HPT 0 130.0
HPT 1 4.97604790419
HPT 2 -1.14942528736
HPT 3 -1.75247524752
HPT 4 -2.82926829268
HPT 5 -2.38596491228
HPT 6 -2.8
HPT 7 -2.07692307692
HPT 8 -2.33333333333
HPT 9 -1.98039215686
HPT 10 -1.8125
AMANAH 0 6.11148648649
AMANAH 1 10.1903485255
AMANAH 2 19.1948340746
AMANAH 3 12.6993377483
AMANAH 4 17.0506024096
AMANAH 5 -12.1267566048
AMANAH 6 -16.0
AMANAH 7 -8.85456039441
AMANAH 8 18.8625218914
AMANAH 9 21.5539513678
AMANAH 10 23.003021148


In [78]:
# stock[sym].loc[stock[sym]['tf30'] == 6]['spread[sym]']
for r in range(11):
#     stock['AMANAH'] = stock['AMANAH'].assign(ln_A = stock[sym].loc[stock[sym]['tf30'] == r ]['spread[sym]'].mean())
    stock['AMANAH'] = stock['AMANAH'].assign(previous = stock[sym].loc[stock[sym]['tf30'] == r]['spread'].mean())

IndentationError: unexpected indent (<ipython-input-78-efaa6b57fb98>, line 3)

In [11]:
# stock['HPT'].loc[stock['HPT']['tf30'] == 2]['spread'].mean()

dfall

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,tomorrow,typeBO,yesterday,previous,avg,predict
699,"[126, 125, 124, 123, 122, 99100, 104500, 30000...",2017-02-16,1.487239e+09,9300.0,0.0,0.0,09:58:34,None,Buy,2017-02-15,-1.657143,0.428571,0.263158
700,"[126, 125, 124, 123, 122, 129100, 104500, 3000...",2017-02-16,1.487239e+09,30000.0,0.0,0.0,09:58:36,None,Bid is slot 0,2017-02-15,-1.657143,0.428571,0.263158
701,"[126, 125, 124, 123, 122, 129100, 104500, 3000...",2017-02-16,1.487239e+09,2000.0,0.0,0.0,09:59:03,None,Offer Not slot 0,2017-02-15,-1.657143,0.428571,0.263158
702,"[126, 125, 124, 123, 122, 138100, 104500, 3000...",2017-02-16,1.487239e+09,9000.0,0.0,0.0,09:59:04,None,Bid is slot 0,2017-02-15,-1.657143,0.428571,0.263158
703,"[127, 126, 125, 124, 123, 10000, 138100, 10450...",2017-02-16,1.487239e+09,-5500.0,1.0,0.0,09:59:25,None,Bid is slot 0,2017-02-15,-1.657143,0.428571,0.263158
704,"[127, 126, 125, 124, 123, 10000, 138100, 10450...",2017-02-16,1.487239e+09,2000.0,1.0,0.0,09:59:45,None,Offer Not slot 0,2017-02-15,-1.657143,0.428571,0.263158
705,"[127, 126, 125, 124, 123, 10000, 138100, 10450...",2017-02-16,1.487239e+09,8000.0,1.0,0.0,09:59:50,None,Offer Not slot 0,2017-02-15,-1.657143,0.428571,0.263158


In [9]:
dfall = pd.DataFrame()

# for row in dfall.itertuples(index=True, name='Pandas'):
#     print getattr(row, "c1"), getattr(row, "c2")


# for n in range(11):
if n == 0:
    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n+10) & (stock['HPT']['Date'] == yesterday)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n+1)  & (stock['HPT']['Date'] == today)]['spread'].mean())

    dfall = dfall.append(dftest)


elif n == 10:
    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock[sym].loc[(stock['HPT']['tf30'] == n) & (stock[sym]['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n-9)  & (stock['HPT']['Date'] == tomorrow)]['spread'].mean())
    dfall = dfall.append(dftest)

else:
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n+1)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dfall = dfall.append(dftest)

In [67]:
dfall1 = pd.DataFrame()
n=0
# for n in range(11):
if n == 0:
    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n+10) & (stock['HPT']['Date'] == yesterday)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n+1)  & (stock['HPT']['Date'] == today)]['spread'].mean())

    dfall1 = dfall1.append(dftest)

In [21]:
dfall2

NameError: name 'dfall2' is not defined

In [20]:
dfall3 = dfall3.drop('typeBO',axis=1)

NameError: name 'dfall3' is not defined

In [116]:
dfall4 = pd.get_dummies(dfall4)

In [117]:
result = pd.concat([dfall3, dfall4], axis=1)
result

,Timestamp,difBO,spread,tf30,time,previous,avg,predict,Bid Not slot 0,Bid is slot 0,Buy,Offer Not slot 0,Offer is slot 0,Sell
699,1.487239e+09,9300.0,0.0,0.0,09:58:34,-1.657143,0.428571,0.263158,0.0,0.0,1.0,0.0,0.0,0.0
700,1.487239e+09,30000.0,0.0,0.0,09:58:36,-1.657143,0.428571,0.263158,0.0,1.0,0.0,0.0,0.0,0.0
701,1.487239e+09,2000.0,0.0,0.0,09:59:03,-1.657143,0.428571,0.263158,0.0,0.0,0.0,1.0,0.0,0.0
702,1.487239e+09,9000.0,0.0,0.0,09:59:04,-1.657143,0.428571,0.263158,0.0,1.0,0.0,0.0,0.0,0.0
703,1.487239e+09,-5500.0,1.0,0.0,09:59:25,-1.657143,0.428571,0.263158,0.0,1.0,0.0,0.0,0.0,0.0
704,1.487239e+09,2000.0,1.0,0.0,09:59:45,-1.657143,0.428571,0.263158,0.0,0.0,0.0,1.0,0.0,0.0
705,1.487239e+09,8000.0,1.0,0.0,09:59:50,-1.657143,0.428571,0.263158,0.0,0.0,0.0,1.0,0.0,0.0
706,1.487239e+09,10000.0,1.0,1.0,10:00:01,0.428571,0.263158,1.487805,0.0,0.0,0.0,0.0,1.0,0.0
707,1.487239e+09,-2000.0,1.0,1.0,10:01:03,0.428571,0.263158,1.487805,1.0,0.0,0.0,0.0,0.0,0.0
708,1.487239e+09,600.0,1.0,1.0,10:01:54,0.428571,0.263158,1.487805,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
if n == 10:
    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock[sym].loc[(stock['HPT']['tf30'] == n) & (stock[sym]['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n-9)  & (stock['HPT']['Date'] == tomorrow)]['spread'].mean())
    dfall = dfall.append(dftest)

In [79]:
n=9
if 0<n<10:
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n+1)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dfall1 = dfall1.append(dftest)

In [11]:
dfall

['OTO', 'HPT', 'AMANAH']

In [14]:
yesterday = "2017-02-14"
today = "2017-02-15"
tomorrow = "2017-02-16"

for sym in symbols:
    dfall[sym] = pd.DataFrame()
    for n in range(11):
        if n == 0:
            # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
            dftest = stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]
            dftest = dftest.assign(previous = stock[sym].loc[(stock[sym]['tf30'] == n+10) & (stock[sym]['Date'] == yesterday)]['spread'].mean())
            dftest = dftest.assign(avg = stock[sym].loc[(stock[sym]['tf30'] == n)  & (stock[sym]['Date'] == today)]['spread'].mean())
            dftest = dftest.assign(predict = stock[sym].loc[(stock[sym]['tf30'] == n+1)  & (stock[sym]['Date'] == today)]['spread'].mean())

            dfall[sym] = dfall[sym].append(dftest)


        elif n == 10:
            # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
            dftest = stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]
            dftest = stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]
            dftest = dftest.assign(previous = stock[sym].loc[(stock[sym]['tf30'] == n-1) & (stock[sym]['Date'] == today)]['spread'].mean())
            dftest = dftest.assign(avg = stock[sym].loc[(stock[sym]['tf30'] == n)  & (stock[sym]['Date'] == today)]['spread'].mean())
            dftest = dftest.assign(predict = stock[sym].loc[(stock[sym]['tf30'] == n-9)  & (stock[sym]['Date'] == tomorrow)]['spread'].mean())
            dfall[sym] = dfall[sym].append(dftest)

        else:
            dftest = stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]                          
            dftest = dftest.assign(previous = stock[sym].loc[(stock[sym]['tf30'] == n-1) & (stock[sym]['Date'] == today)]['spread'].mean())
            dftest = dftest.assign(avg = stock[sym].loc[(stock[sym]['tf30'] == n)  & (stock[sym]['Date'] == today)]['spread'].mean())
            dftest = dftest.assign(predict = stock[sym].loc[(stock[sym]['tf30'] == n+1)  & (stock[sym]['Date'] == today)]['spread'].mean())
            dfall[sym] = dfall[sym].append(dftest)

In [221]:
# stock['AMANAH'].loc[(stock['AMANAH']['tf30'] == n) & (stock['AMANAH']['Date'] == today)]
stock[sym].loc[(stock[sym]['tf30'] == n) & (stock[sym]['Date'] == today)]

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,typeBO
19425,"[160, 159, 158, 157, 156, 64700, 153200, 21000...",2017-02-15,1.487174e+09,8000.0,-20.0,10.0,16:00:03,Offer is slot 0
19426,"[160, 159, 158, 157, 156, 64700, 153200, 21000...",2017-02-15,1.487174e+09,100.0,-20.0,10.0,16:00:09,Offer Not slot 0
19427,"[160, 159, 158, 157, 156, 64700, 153200, 21000...",2017-02-15,1.487174e+09,100.0,-20.0,10.0,16:00:12,Offer Not slot 0
19428,"[160, 159, 158, 157, 156, 64700, 153200, 21000...",2017-02-15,1.487174e+09,600.0,-20.0,10.0,16:00:18,Offer is slot 0
19429,"[160, 159, 158, 157, 156, 64700, 153200, 21000...",2017-02-15,1.487174e+09,500.0,-20.0,10.0,16:00:25,Offer Not slot 0
19430,"[160, 159, 158, 157, 156, 64700, 153200, 21000...",2017-02-15,1.487174e+09,1500.0,-20.0,10.0,16:00:27,Offer is slot 0
19431,"[160, 159, 158, 157, 156, 64700, 153200, 21000...",2017-02-15,1.487174e+09,-1500.0,-20.0,10.0,16:00:27,Offer Not slot 0
19432,"[160, 159, 158, 157, 156, 64800, 153200, 21000...",2017-02-15,1.487174e+09,100.0,-20.0,10.0,16:00:27,Bid is slot 0
19433,"[160, 159, 158, 157, 156, 64800, 140200, 21000...",2017-02-15,1.487174e+09,-13000.0,-20.0,10.0,16:00:33,Bid Not slot 0
19434,"[161, 160, 159, 158, 157, 13000, 64800, 140200...",2017-02-15,1.487174e+09,-2800.0,-19.0,10.0,16:00:33,Bid is slot 0


In [16]:
dfall['OTO']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,tomorrow,typeBO,yesterday,previous,avg,predict
3465,"[720, 715, 710, 705, 700, 35000, 57600, 45300,...",2017-02-15,1.487153e+09,-5000.0,-1.0,0.0,09:59:05,2017-02-16,Buy,2017-02-14,-2.802708,-0.750000,-4.605882
3466,"[720, 715, 710, 705, 700, 35000, 57600, 45300,...",2017-02-15,1.487153e+09,10000.0,-1.0,0.0,09:59:32,2017-02-16,Offer is slot 0,2017-02-14,-2.802708,-0.750000,-4.605882
3467,"[720, 715, 710, 705, 700, 35000, 57600, 45300,...",2017-02-15,1.487153e+09,-10000.0,-1.0,0.0,09:59:32,2017-02-16,Offer Not slot 0,2017-02-14,-2.802708,-0.750000,-4.605882
3468,"[720, 715, 710, 705, 700, 40000, 57600, 45300,...",2017-02-15,1.487153e+09,5000.0,-1.0,0.0,09:59:46,2017-02-16,Bid is slot 0,2017-02-14,-2.802708,-0.750000,-4.605882
3469,"[720, 715, 710, 705, 700, 40000, 57600, 45300,...",2017-02-15,1.487153e+09,4000.0,-1.0,0.0,09:59:47,2017-02-16,Offer Not slot 0,2017-02-14,-2.802708,-0.750000,-4.605882
3470,"[720, 715, 710, 705, 700, 40000, 57600, 45300,...",2017-02-15,1.487153e+09,50000.0,-1.0,0.0,09:59:47,2017-02-16,Offer Not slot 0,2017-02-14,-2.802708,-0.750000,-4.605882
3471,"[725, 720, 715, 710, 705, 36000, 40000, 57600,...",2017-02-15,1.487153e+09,12700.0,0.0,0.0,09:59:59,2017-02-16,Buy,2017-02-14,-2.802708,-0.750000,-4.605882
3472,"[725, 720, 715, 710, 705, 36000, 40000, 57600,...",2017-02-15,1.487153e+09,115700.0,0.0,0.0,09:59:52,2017-02-16,Offer is slot 0,2017-02-14,-2.802708,-0.750000,-4.605882
3473,"[725, 720, 715, 710, 705, 36000, 40000, 57600,...",2017-02-15,1.487153e+09,600.0,0.0,1.0,10:00:07,2017-02-16,Offer Not slot 0,2017-02-14,-0.750000,-4.605882,-4.654930
3474,"[720, 715, 710, 705, 700, 40000, 57600, 45300,...",2017-02-15,1.487153e+09,-12700.0,-1.0,1.0,09:58:41,2017-02-16,Sell,2017-02-14,-0.750000,-4.605882,-4.654930


In [188]:
tomorrow = "2017-02-15"
today = "2017-02-14"
n=10
if n == 10:

    # stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
    dftest = stock['HPT'].loc[(stock['HPT']['tf30'] == n) & (stock['HPT']['Date'] == today)]                          
    dftest = dftest.assign(previous = stock['HPT'].loc[(stock['HPT']['tf30'] == n-1) & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(avg = stock['HPT'].loc[(stock['HPT']['tf30'] == n)  & (stock['HPT']['Date'] == today)]['spread'].mean())
    dftest = dftest.assign(predict = stock['HPT'].loc[(stock['HPT']['tf30'] == n-9)  & (stock['HPT']['Date'] == tomorrow)]['spread'].mean())

In [204]:
stock['HPT']

,BidOffer,Date,Timestamp,difBO,spread,tf30,time,typeBO
0,"[130, 129, 128, 127, 126, 59800, 134100, 26700...",2017-02-14,1.487066e+09,20000.0,130.0,0.0,09:57:03,Offer Not slot 0
1,"[130, 129, 128, 127, 126, 109800, 134100, 2670...",2017-02-14,1.487066e+09,50000.0,130.0,1.0,10:00:20,Bid is slot 0
2,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,200.0,130.0,1.0,10:02:29,Bid Not slot 0
3,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,100000.0,130.0,1.0,10:02:40,Bid Not slot 0
4,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:02:55,Offer Not slot 0
5,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:10,Offer Not slot 0
6,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:20,Bid Not slot 0
7,"[130, 129, 128, 127, 126, 109800, 134100, 2672...",2017-02-14,1.487067e+09,10000.0,130.0,1.0,10:03:20,Offer Not slot 0
8,"[130, 129, 128, 127, 126, 50800, 134100, 26720...",2017-02-14,1.487067e+09,-59000.0,0.0,1.0,10:03:22,Sell
9,"[129, 128, 127, 126, 125, 134100, 267200, 1780...",2017-02-14,1.487067e+09,120200.0,-1.0,1.0,09:56:11,Sell


In [169]:
# stock[sym].loc[stock[sym]['tf30'] == 1]['spread[sym]'].mean()
dftest = stock['HPT'].loc[stock['HPT']['tf30'] == 2]                                
dftest = dftest.assign(previous = stock['HPT'].loc[stock['HPT']['tf30'] == 1]['spread'].mean())
dftest = dftest.assign(avg = stock['HPT'].loc[stock['HPT']['tf30'] == 2]['spread'].mean())
dftest = dftest.assign(predict = stock['HPT'].loc[stock['HPT']['tf30'] == 3]['spread'].mean())

In [126]:
dftest2 = stock['HPT'].loc[stock['HPT']['tf30'] == 3]
previoustest2 = stock[['HPT'].loc[stock[['HPT']['tf30'] == 2]['spread'].mean()
avgtest2 = stock[['HPT'].loc[stock[['HPT']['tf30'] == 3]['spread'].mean()
predicttest2 = stock[['HPT'].loc[stock[['HPT']['tf30'] == 4]['spread'].mean()

dftest2 = dftest2.assign(previous = previoustest2)
dftest2 = dftest2.assign(avg = avgtest2)
dftest2 = dftest2.assign(predict = predicttest2)

In [239]:
yesterday = "2017-02-14"
today = "2017-02-15"
tomorrow = "2017-02-15"

if n==9:
    dftest2 = stock['HPT'].loc[(stock['HPT']['tf30'] == 9) & (stock['HPT']['Date'] == today)]
    previoustest2 = stock[['HPT'].loc[stock[['HPT']['tf30'] == 8]['spread'].mean()
    avgtest2 = stock['HPT'].loc[stock['HPT']['tf30'] == 9]['spread'].mean()
    predict2 = stock['HPT'].loc[(stock['HPT']['tf30'] == 1 ) & (stock['HPT']['Date'] ==tomorrow)]['spread'].mean()

dftest2 = dftest2.assign(previous = round(previoustest2), 0)
dftest2 = dftest2.assign(avg = round(avgtest2), 0)
dftest2 = dftest2.assign(predict = round(predicttest2), 0)

SyntaxError: invalid syntax (<ipython-input-239-cf0f9aec05de>, line 8)

In [141]:
dftest
dftest2

dftest = dftest.append(dftest2)

In [18]:
for n in range(11):
    print round(stock['HPT'].loc[(stock['HPT']['tf30'] == n ) & (stock['HPT']['Date'] == "2017-02-14")]['spread'].mean(),0)

# avgtest2 = stock['HPT'].loc[stock['HPT']['tf30'] == 9]['spread'].mean()

130.0
6.0
-2.0
-1.0
-1.0
-1.0
nan
-1.0
-1.0
-1.0
-2.0


In [19]:
result

NameError: name 'result' is not defined